In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten
from keras.models import load_model, Model
from keras.layers import Conv2D, MaxPooling2D, Input
from keras.activations import relu
from keras.losses import SparseCategoricalCrossentropy
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from keras.applications import ResNet152




In [2]:
!pip install kaggle
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

!chmod 600 /root/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d sujaykapadnis/monkeypox-skin-images-dataset-msid

 60% 33.0M/54.6M [00:00<00:00, 115MB/s] 
100% 54.6M/54.6M [00:00<00:00, 138MB/s]


In [5]:
!unzip -q monkeypox-skin-images-dataset-msid


In [6]:
dataPath = '/content/Monkeypox Skin Images Dataset (MSID)/Monkeypox Skin Image Dataset/Monkeypox Skin Image Dataset'
img_height = 224
img_width = 224
dataGen = ImageDataGenerator(
    rescale=1/255.,
    validation_split = 0.2
)
TrainData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='training',
)
validationData = dataGen.flow_from_directory(
    dataPath,
    target_size=((img_height, img_width)),
    class_mode='categorical',
    batch_size=32,
    subset='validation',
)




Found 618 images belonging to 4 classes.
Found 152 images belonging to 4 classes.


In [7]:

baseModel = ResNet152(include_top=False, weights='imagenet', input_shape=((img_height, img_width, 3)))

for layer in baseModel.layers:
  layer.trainable = False

input_layer = Input(shape=(224,224,3))
modelOuput = baseModel(input_layer) # Pass the input through the VGG16 base model
fOutput = Flatten()(modelOuput) # Flatten the output and add additional fully connected layers
dense_1 = Dense(512, activation='relu')(fOutput)
dense2 = Dense(256, activation='relu')(dense_1)
output_layer = Dense(4, activation='softmax')(dense2)

model = Model(inputs=input_layer, outputs=output_layer)
# Compile the model

234698864/234698864 [==============================] - 3s 0us/step


In [8]:

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 512)               51380736  
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 4)                 1028      
                                                                 
Total params: 109884036 (419.17 MB)
Trainable params: 5151309

In [9]:

history = model.fit(
    TrainData,
    epochs=50,  # adjust the number of epochs as needed
    validation_data=validationData
)

Epoch 1/50
20/20 [==============================] - 28s 741ms/step - loss: 11.6658 - accuracy: 0.3107 - val_loss: 3.1693 - val_accuracy: 0.3816
Epoch 2/50
20/20 [==============================] - 6s 317ms/step - loss: 3.0088 - accuracy: 0.3738 - val_loss: 1.6971 - val_accuracy: 0.4474
Epoch 3/50
20/20 [==============================] - 7s 327ms/step - loss: 1.5048 - accuracy: 0.4725 - val_loss: 1.2937 - val_accuracy: 0.4737
Epoch 4/50
20/20 [==============================] - 7s 324ms/step - loss: 1.2426 - accuracy: 0.4871 - val_loss: 1.3712 - val_accuracy: 0.4408
Epoch 5/50
20/20 [==============================] - 7s 334ms/step - loss: 1.1994 - accuracy: 0.5097 - val_loss: 1.2063 - val_accuracy: 0.4671
Epoch 6/50
20/20 [==============================] - 7s 327ms/step - loss: 1.1659 - accuracy: 0.5210 - val_loss: 1.1415 - val_accuracy: 0.5197
Epoch 7/50
20/20 [==============================] - 7s 337ms/step - loss: 1.1245 - accuracy: 0.5761 - val_loss: 1.4369 - val_accuracy: 0.3947
Epoc

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
